※ このNotebookでは、動作確認のためにNotebook経由でVCノードの起動と削除を実施します。


# 初期設定

## 環境設定ファイル `vcp_config.yml` の編集

VCP SDKを使用するには、VCコントローラの情報と、VMwareのログイン情報が必要となります。

設定は `vcpsdk/vcpsdk/config/vcp_config.yml` に記述します。以下の内容を記述してください。

- VCコントローラのIPアドレス
- VCコントローラの識別名
- VMwareのログインユーザ名の vaultへの格納先
- VMwareのログインパスワードの vaultへの格納先


```
#
# VCコントローラ情報
#
vcc:
    host: x.x.x.x #VCコントローラ IPアドレス
    name: xxxx  #VCコントローラ識別名
#
# VMware の設定
#
vmware:
    user: "vault://cubbyhole/vmware/user"
    password: "vault://cubbyhole/vmware/password"
    private_network: "default"
    
```

VMwareのログインユーザ名・パスワードなどの認証情報は、HashiCorp Vaultを使用して管理します。Valutについての詳細は、[秘密情報管理サーバの利用](/jupyter/notebooks/hands-on/vcp-tutorial/102-%E7%A7%98%E5%AF%86%E6%83%85%E5%A0%B1%E7%AE%A1%E7%90%86%E3%82%B5%E3%83%BC%E3%83%90%E3%81%AE%E5%88%A9%E7%94%A8.ipynb)を参考にしてください。


In [ ]:
# vcp_config.yml の編集用ボタン表示
!cp /notebooks/notebook/vcpsdk/vcpsdk/config/sample-vcp_config.yml /notebooks/notebook/vcpsdk/vcpsdk/config/vcp_config.yml


import os

config_url = "/{}/edit/vcpsdk/vcpsdk/config/vcp_config.yml".format(os.environ['SUBDIR'])
from IPython.core.display import HTML

style= "text-decoration: none;\
padding: 4px 8px;border: solid 2px #ddf; color: #fff;\
font-size: 20px;\
font-weight: bold;\
background-color: rgb(100, 149, 237);"
HTML('<a style="{}" target="new" href="{}">vcp_config.ymlを編集</a>'.
     format(style, config_url))

## アクセストークンの発行

VCP SDKを利用するためにはVCP REST APIアクセストークンが必要となります。  
まず、管理者用 Web UI にログインして、新規にアクセストークンを発行してください。

[<font size="+1">**管理者用 Web UI ログイン**</font>](https://tokyo01.vcloud.nii.ac.jp/vcp/v1/ui/occmgr/menu)

管理者用Web UIの利用方法の詳細についてはユーザーズマニュアルをご参照ください。

## Vaultへのクレデンシャル情報の設定

以下のクレデンシャル情報を変更し、Jupyter Notebook サーバのTerminalに張り付け実行してください。  
Vaultではアクセストークンごとに格納先が異なるので、アクセストークンを使用する利用者がそれぞれ実行してください。

```
cat << 'EOF' > post_vault.sh
#!/bin/bash

VCP_TOKEN=VCPマネージャで発行したアクセストークン
VMWARE_USER=VMwareのログインユーザ名
VMWARE_PASSWORD=VMwareのログインパスワード

curl -H "X-Vault-Token: ${VCP_TOKEN}" -X POST https://133.8.140.6:8443/v1/cubbyhole/vmware/user -d "{\"value\":\"${VMWARE_USER}\"}"
curl -H "X-Vault-Token: ${VCP_TOKEN}" -X POST https://133.8.140.6:8443/v1/cubbyhole/vmware/password -d "{\"value\": \"${VMWARE_PASSWORD}\"}"
EOF

sh post_vault.sh
rm post_vault.sh
```

## ssh公開鍵の作成

Jupyter Notebook サーバからVCノードに ssh 接続するために、ssh キーペアを作成しておきます。

In [ ]:
# 動作確認用SSHキーペア作成
!mkdir -p ~/.ssh
!test -f ~/.ssh/id_rsa || ssh-keygen -t rsa -f ~/.ssh/id_rsa -N ""

## `vcp_flavor.yml`

flavor (CPU数, メモリサイズ など、VMの仮想ハードウェアの設定を抽象化したもの) を定義するファイルです。  
設定は `vcpsdk/vcpsdk/config/vcp_flavor.yml` に記述します。

デフォルトで設定された定義がすでにありますが、必要であれば利用者ごとにカスタマイズしてご使用ください。

記述方法は各クラウドプロバイダ毎に違います。詳細は[VCP SDK マニュアル](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=29239877)の以下の項目をご確認ください。  
       
- 1.6.3. flavor
- 2.15. SDK plugins for VMware (vcpsdk/plugins/vmware)

※ 上記マニュアルのpluginの説明のように、VCP SDKでのspec、disk specの仕様に関しましても各クラウドプロバイダ毎に違いがあります。

※ この動作確認では、デフォルトのフレーバー設定を使用して進めます。  


# VCノードの作成と削除

## VCP SDK の初期化

VCP SDK を使用ために、はじめにアクセストークンを使用して VCP SDKを初期化してください。  
次のセルを実行すると、VCコントローラのアクセストークンの入力枠が表示されます。  
アクセストークンの値を入力し、Enterキーを押すことで初期化が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCP SDKの初期化
vcp = VcpSDK(
    vcc_access_token,  # VC Controllerのアクセストークン
)

vcp.version()


### もしエラーが発生したら...

- `Exception: ('VCP', 'config vc failed')` ... 使われたアクセストークンが正しくない
- `Exception: ('VCP', 'server error')` ... VCコントローラのIPアドレス設定が正しくないか、ネットワーク接続エラーが発生している

## VCノードの起動

次のセルを実行し、動作確認用のVCノードを1個作成します。  
クラウドのインスタンスを起動してVCノード用の初期設定を行うため、実行完了まで数分程度を要します。

In [ ]:
# UnitGroupの作成
unit_group = vcp.create_ugroup(
    'testunit'    # UnitGroupの名前
)

# flavorの指定
spec = vcp.get_spec('vmware', 'small')

#
# 設定例
#
#spec.num_nodes = 1
#spec.ip_addresses = ['起動するnodeの静的なIPアドレス']

# docker run のオプション指定 （-v -e）
#spec.params_v = ['/opt:/opt']
#spec.params_e = ['USER_NAME=test']

# VMware 依存
#   https://www.terraform.io/docs/providers/vsphere/index.html
# spec.num_cpus = 4
# spec.memory = 1024 # MB
# spec.disk_size = 80
# ※ テンプレートを使用する場合にテンプレートのディスクサイズより小さい disk_size を指定するとエラーになります。
#    テンプレートより大きい disk_size を指定した場合は成功しますが、パーティションの拡張や追加はされないので、
#    拡張したディスク領域を使用するためには、別途パーティションの拡張や追加の作業が必要になります。

# クラウドイメージ(テンプレートとするVMWareの仮想マシンの名前)
# spec.cloud_image = 'vcp-template-xxxx'

# base containerにssh loginするためのssh公開鍵情報を設定
# spec.set_ssh_pubkey('/home/jovyan/.ssh/id_rsa.pub')

spec.ip_addresses = ['y.y.y.y']  # 使用するIPアドレスを指定ください
spec.disk_size = 80
spec.cloud_image = 'vcp-template'

spec.set_ssh_pubkey('/home/jovyan/.ssh/id_rsa.pub')

unit = unit_group.create_unit(
    'testserver',    # Unit名の指定
    spec
)


### もしエラーが発生したら...

クラウドのクレデンシャル情報が正しくない場合、以下のようなエラーが発生します。  
Valut に設定した内容を確認してください。

```
*** HAS ERROR:
---
2019/06/12 05:09:13 UTC: * provider.aws: InvalidClientTokenId: The security token included in the request is invalid.
---
```

## 起動したVCノードの確認

次のセルを実行し、起動したVCノードを確認してください。

In [ ]:
unit.df_nodes()

## VCノードへのsshログイン

JupyterNotebook の Terminal 機能を用いて、作成したVCノードにsshログインすることができます。  
VCノードのIPアドレスは上記の `unit.df_nodes()` の出力に含まれています。

```
$ ssh -i ~/.ssh/id_rsa root@{cloud_instance_address}
```

## GrafanaでVCノードの利用状況を確認する

[Grafana](http://133.8.140.6:3000/d/vcp/vcp-metrics?refresh=5s) からダッシュボードを開いてください。

最初にログイン画面が表示されるので ユーザ名、パスワードにそれぞれ `admin`, `admin`を入力してください。 

ログインすると負荷状況のグラフが表示されます。左半分のグラフがBaseコンテナの負荷状況で、右半分がアプリケーションコンテナの負荷状況のグラフです。

アプリケーションコンテナを立ち上げると右側にもグラフが表示されます。(stress コマンドを実行)

In [ ]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
ip_address = unit_group.find_ip_addresses(node_state='RUNNING')[0] # 今は１つのVCノードのみ起動しているので [0] で最初の要素を取り出す

import os
ssh_private_key = os.path.expanduser('~/.ssh/id_rsa')

# 60秒間だけ CPU x 2 とメモリ 128 MB を消費する
!ssh -i {ssh_private_key} -l root {ip_address} \
    /usr/local/bin/docker run -td --rm --name stress-0 polinux/stress \
    stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

## VCノードの削除
次のセルを実行し、動作確認に使用したクラウドインスタンスを削除してください。  
実行完了まで数分程度を要します。

In [ ]:
unit_group.cleanup()

In [ ]:
vcp.df_ugroups()